# Some examples of using Paicos

This script shows how to

- load data
- make projections and slices
- save them as an 'ArepoImage'
- how to convert the Arepo data from comoving code units to physical values in various unit systems


## Compilation
The first step is to compile the code, this only needs to be done the first time you use Paicos (uncomment all four lines and replace the path to your own cloned version)

In [ ]:
#%%bash
#cd /llust21/berlok/paicos
#make clean
#make

## Loading arepo snapshots

We load a zoom factor 12 galaxy cluster simulation below

In [ ]:
import paicos as pa
import numpy as np
simfolder = '/llust21/cosmo-plasm/zoom-simulations/halo_0003/adiabatic-mhd/zoom12/'

# A snapshot object
snap = pa.Snapshot(simfolder + 'output', 130)

# The center of the most massive Friends-of-friends group in the simulation
center = snap.Cat.Group['GroupPos'][0]

# The available fields for a PartType can be found as shown below
keys = snap.info(0)

We can look at some of snaps attributes:

In [ ]:
snap['0_Density']

In [ ]:
print(snap.age)
print(snap.lookback_time)
print(snap.converter.cosmo)
snap.box_size
# print(snap.Config)
# print(snap.Parameters)
# print(snap.Header)
# print(snap.z)


Loading of data can be done using function calls or by trying to access them explicitly.

In [ ]:
# Load some variables from the PartType 0 (gas variables) 

# You can explicitly load using function call:
snap.load_data(0, 'Coordinates')
snap['0_Coordinates']

# But is much easier to just do it like this:
snap['0_Density']
snap['0_MagneticField']

# snap
snap

This loads a PaicosQuantity (basically a subclass of an astropy quantity). Here we can see the units used in the simulation. small_h and small_a are the reduced Hubble parameter and the scale factor, respectively. These quantities have the following useful methods:

In [ ]:
# Unit conversion
rho = snap['0_Density']
print('rho[0] in CGS:\t', rho[0].cgs)
print('rho[0] in SI:\t', rho[0].si)
print("rho[0] in 'astro' units:\t", rho[0].astro)
print("rho[0] in Msun/au^3:\t", rho[0].to('Msun/au3'), '\n\n')

# Get rid of h factors
print('rho[0] without h:\t', rho[0].no_small_h)

# Get rid of both a and h factors
print('rho[0] without a and h:\t', rho[0].to_physical, '\n\n')

# Get a label for use in plots
print(rho.label(r'\rho'))
print(rho.astro.label(r'\rho'))

Please note that all these methods return a new object without modifying the original data. Modification can be done by overwriting, e.g., like this:

In [ ]:
rho = rho.to_physical
rho

You can ask astropy to figure out other ways of writing units, e.g.:

In [ ]:
rho.unit.compose()

This allows us to do the following:

In [ ]:
rho.to("arepo_density")

You can also automatically calculate derived variables, e.g.:

In [ ]:
snap['0_Volume']
snap['0_TemperaturesTimesMasses']
snap['0_VelocityCurvature']

## Making projections

We now use the Paicos projector class, the 'widths' vector is the size of the considered box in x,y,z coordinates. This box is centered at 'center' vector.

The direction can be set to 'x', 'y' or 'z'. If the direction is 'z' (as below) then widths[2] is the depth of the projection and the 2D returned array is in the xy plane.

npix is the number of pixels in the horizontal direction of the image. The width/height ratio should be such that $$npix*height/width$$ is an integer.

In [ ]:
widths = [26000, 13000, 10000]
projector = pa.Projector(snap, center, widths, 'z', npix=2048)

We can call the project_variable method as below. This method can take a number of standard strings (which then internally calls the get_variable function, see further details below) or it can take an array. Both methods are shown below.

In [ ]:
Masses = projector.project_variable('0_Masses')
Volumes = projector.project_variable(snap['0_Volume'])
rho = Masses/Volumes

rho

We can now plot the projected density

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.rc('image', origin='lower', cmap='RdBu_r', interpolation='None')
plt.imshow(rho, cmap='YlGnBu', extent=projector.extent.value, norm=LogNorm())
# plt.savefig('Z24_snap130_wide_projection.pdf', dpi=2000, bbox_inches='tight')

## Making slices

Next, we will take a look at making a slice through the simulation. The width is by definition zero, and the user has to set this explicitly by setting a zero in the 'widths' vector. Below we show a slice of density, comparing with the projected density.

In [ ]:
widths = [26000, 13000, 0]
slicer = pa.Slicer(snap, center, widths, 'z', npix=2048)

In [ ]:
plt.figure(1)
plt.clf()
fix, axes = plt.subplots(nrows=2)

# Slice by passing an array
rho_slice = slicer.slice_variable(snap['0_Density'])

# Slice by passing a string (see snap.info(0) for the available strings)
rho_slice = slicer.slice_variable('0_Density')

# Now plot slice and projection next to each other
axes[0].imshow(rho_slice.to_physical, norm=LogNorm())
axes[1].imshow(rho, norm=LogNorm())
axes[0].set_title('Slice')
axes[1].set_title('Projected')
for ii in range(2):
    axes[ii].set_axis_off()
# plt.savefig('halo3_Z12_slice_projec_comparison.pdf', dpi=2000, bbox_inches='tight')

We can also make slices of other variables. The Slicer object stores the required information (indices of the Voronoi cells closest to the image grid points), so the computing time needed for making additional slices is neglibible.

Let us for instance consider the enstrophy which gives an indication of the amount of turbulence in the galaxy cluster.
It is defined as

1/2|∇×v|²

and can be found from the 'VelocityGradient' field (the 3x3 tensor of velocity derivatives, ∂ᵢvⱼ). This is done internally below:

In [ ]:
extent = slicer.extent.to('Mpc')

plt.imshow(slicer.slice_variable('0_Enstrophy'),
           extent=extent.value,
           norm=LogNorm())

## Storing image data

The computing time for slices, and in particular, projections, is often quite long. It is therefore convenient to be able to store the image data so that this step is de-coupled from the often many matplotlib iterations.

Below I illustrate how to save an Arepo image, created using either a Projector or Slicer object.

In [ ]:
image_file = pa.ArepoImage(slicer, basedir=pa.root_dir + '/data/',
                           basename='test_arepo_image_format')

image_file.save_image('Density', slicer.slice_variable('0_Density'))
image_file.save_image('Enstrophy', slicer.slice_variable('0_Enstrophy'))

image_file.finalize()

The constructed file is found at:

In [ ]:
image_file.filename

Now lets open this image and looks at its contents:

In [ ]:
import h5py
f = h5py.File(image_file.filename, 'r')

In [ ]:
list(f.keys())

Here 'Config', 'Header', 'Parameters' are groups copied over from the snapshot file used to create the image (.0.hdf5 when there are multiple files). 'Density' and 'Enstrophy' is 2D array with the saved images. The group 'image_info' contains essential information about the image, namely:

In [ ]:
print(f['image_info'].keys())
print(f['image_info'].attrs.keys())

We can plot the image and use 'image_info' to get the extent of the image.

In [ ]:
im = plt.imshow(f['Density'], extent=f['image_info']['extent'], norm=LogNorm())
cbar = plt.colorbar(im, fraction=0.025, pad=0.04)

## Getting the units right

The plot above is still in comoving code units, we can use the ImageReader class to automatically get the image data in the form of PaicosQuantities (i.e. with units and in-built methods for manipulation). All the relevant information is stored in the image file, e.g.:

In [ ]:
dict(f['Density'].attrs)

In [ ]:
im = pa.ImageReader(basedir=pa.root_dir + '/data/', snapnum=130,
                 basename='test_arepo_image_format')

extent = im.extent.to('Mpc').to_physical
rho = im['Density'].to_physical
rho = rho.astro
#rho = rho.cgs
image = plt.imshow(rho, extent=extent.value, norm=LogNorm())
cbar = plt.colorbar(image, fraction=0.025, pad=0.04)
cbar.set_label(rho.label('\\rho'))
plt.xlabel(extent.label())
plt.ylabel(extent.label())